In [1]:
pwd

'C:\\Users\\chand\\Desktop\\Projects\\ABIDEII'

In [2]:
import pandas as pd
df = pd.read_csv("ABIDEII_Composite_Phenotypic.csv")
df.head()



,SITE_ID,SUB_ID,NDAR_GUID,DX_GROUP,PDD_DSM_IV_TR,ASD_DSM_5,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,...,ADI_R_C3_TOTAL,ADI_R_C4_REPETITIVE_USE_OBJECTS,ADI_R_C4_HIGHER,ADI_R_C4_UNUSUAL_SENSORY_INTERESTS,ADI_R_C4_TOTAL,ADI_R_D_AGE_PARENT_NOTICED,ADI_R_D_AGE_FIRST_SINGLE_WORDS,ADI_R_D_AGE_FIRST_PHRASES,ADI_R_D_AGE_WHEN_ABNORMALITY,ADI_R_D_INTERVIEWER_JUDGMENT
0,ABIDEII-BNI_1,29006,NaN,1,NaN,NaN,48.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABIDEII-BNI_1,29007,NaN,1,NaN,NaN,41.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABIDEII-BNI_1,29008,NaN,1,NaN,NaN,59.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABIDEII-BNI_1,29009,NaN,1,NaN,NaN,57.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABIDEII-BNI_1,29010,NaN,1,NaN,NaN,45.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df["SUB_ID"] = ["sub-"+str(i) for i in df['SUB_ID']]

In [12]:
def getlableWithID(num):
    for  index,i in enumerate(df['SUB_ID']):
        # print(i)
        if num ==  i:
            return df.iloc[index]['DX_GROUP'],i

In [13]:
a,b = getlableWithID('sub-29273')
type(a),type(b)
a,b

(1, 'sub-29273')

## Final

In [20]:
import os
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Paths
DATASET_ROOT = "D:\\RawData"  # Update this to your dataset path
CSV_FILE = "ABIDEII_Composite_Phenotypic.csv"  # CSV containing metadata
OUTPUT_DIR = "output"  # Where processed images will be saved

# Load CSV
df = pd.read_csv(CSV_FILE)
df["SUB_ID"] = df["SUB_ID"].astype(str)  # Ensure SUB_ID is string

# Create a lookup dictionary {SUB_ID: (DX_GROUP, SITE_ID)}
label_dict = {row["SUB_ID"]: (row["DX_GROUP"], row["SITE_ID"]) for _, row in df.iterrows()}

def getLabelWithID(sub_id):
    """ Returns (DX_GROUP, SITE_ID) for a given patient ID. """
    return label_dict.get(sub_id, (None, None))

def save_slice(image, filepath):
    """ Saves a single image slice in grayscale. """
    plt.imshow(image, cmap='gray')
    plt.axis("off")
    plt.savefig(filepath, dpi=100, bbox_inches='tight', pad_inches=0)
    plt.close()

def process_nifti(nifti_path, sub_id, dataset_name):
    """
    Processes the first available NIfTI file and saves axial, coronal, sagittal slices.
    
    Parameters:
    - nifti_path: Path to .nii/.nii.gz file
    - sub_id: Patient ID
    - dataset_name: Folder name of dataset
    """
    dx_group, site_id = getLabelWithID(sub_id)
    if dx_group is None or site_id is None:
        print(f"⚠️ Skipping {sub_id}: No DX_GROUP or SITE_ID found.")
        return

    # Folder format: ABIDEII-SITEID_SUBID (e.g., ABIDEII-OILH_2_28728)
    patient_folder = f"ABIDEII-{site_id}_{sub_id}"
    save_dir = os.path.join(OUTPUT_DIR, patient_folder)
    os.makedirs(save_dir, exist_ok=True)

    # Load NIfTI image
    try:
        nifti_img = nib.load(nifti_path)
        nifti_data = nifti_img.get_fdata()
    except Exception as e:
        print(f"❌ Error loading {nifti_path}: {e}")
        return

    # Get middle slices
    middle_axial = nifti_data.shape[2] // 2
    middle_sagittal = nifti_data.shape[0] // 2
    middle_coronal = nifti_data.shape[1] // 2

    # Save filenames in correct format
    axial_path = os.path.join(save_dir, f"{patient_folder}_axial.png")
    sagittal_path = os.path.join(save_dir, f"{patient_folder}_sagittal.png")
    coronal_path = os.path.join(save_dir, f"{patient_folder}_coronal.png")

    # Extract and save slices
    save_slice(np.rot90(nifti_data[:, :, middle_axial]), axial_path)  # Axial
    save_slice(np.rot90(nifti_data[middle_sagittal, :, :].T), sagittal_path)  # Sagittal
    save_slice(np.rot90(nifti_data[:, middle_coronal, :].T), coronal_path)  # Coronal

    print(f"✅ Saved images for {patient_folder}")

# Loop through dataset folders
for dataset_name in os.listdir(DATASET_ROOT):
    dataset_path = os.path.join(DATASET_ROOT, dataset_name)
    if not os.path.isdir(dataset_path):
        continue  # Skip files

    for subject in os.listdir(dataset_path):
        subject_path = os.path.join(dataset_path, subject)
        if not os.path.isdir(subject_path) or not subject.startswith("sub-"):
            continue

        sub_id = subject.replace("sub-", "")  # Extract numeric ID

        for session in os.listdir(subject_path):
            session_path = os.path.join(subject_path, session)
            if not os.path.isdir(session_path) or not session.startswith("ses-"):
                continue

            anat_path = os.path.join(session_path, "anat")
            if not os.path.isdir(anat_path):
                continue

            # Pick the FIRST available .nii/.nii.gz file
            nifti_files = [f for f in os.listdir(anat_path) if f.endswith(".nii") or f.endswith(".nii.gz")]
            if nifti_files:
                nifti_file_path = os.path.join(anat_path, nifti_files[0])  # Take only the first one
                process_nifti(nifti_file_path, sub_id, dataset_name)


✅ Saved images for ABIDEII-ABIDEII-BNI_1_29006
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29007
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29008
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29009
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29010
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29011
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29012
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29013
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29014
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29015
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29016
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29017
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29018
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29019
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29020
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29021
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29022
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29023
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29024
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29025
✅ Saved images for ABIDEII-ABIDEII-BNI_1_29026
✅ Saved image

In [21]:
import os
import shutil
import pandas as pd

# Paths
OUTPUT_DIR = "D:\output"  # Folder where images are currently saved
CSV_FILE = "ABIDEII_Composite_Phenotypic.csv"  # Phenotype file

# Load CSV
df = pd.read_csv(CSV_FILE)
df["SUB_ID"] = df["SUB_ID"].astype(str)  # Ensure SUB_ID is string

# Create dictionary {SUB_ID: DX_GROUP}
label_dict = {row["SUB_ID"]: row["DX_GROUP"] for _, row in df.iterrows()}

# Create directories for separation
autism_dir = os.path.join(OUTPUT_DIR, "Autism")
non_autism_dir = os.path.join(OUTPUT_DIR, "Non_Autism")
os.makedirs(autism_dir, exist_ok=True)
os.makedirs(non_autism_dir, exist_ok=True)

# Loop through subject folders
for folder in os.listdir(OUTPUT_DIR):
    folder_path = os.path.join(OUTPUT_DIR, folder)

    # Skip if not a directory
    if not os.path.isdir(folder_path) or folder in ["Autism", "Non_Autism"]:
        continue

    # Extract SUB_ID from folder name
    parts = folder.split("_")
    if len(parts) < 2:
        print(f"Skipping {folder} (Invalid format)")
        continue
    
    sub_id = parts[-1]  # Extract last part as SUB_ID

    # Check DX_GROUP
    dx_group = label_dict.get(sub_id)
    if dx_group is None:
        print(f"Skipping {folder} (SUB_ID not found in phenotype)")
        continue

    # Determine target directory
    target_dir = autism_dir if dx_group == 1 else non_autism_dir

    # Move folder
    shutil.move(folder_path, os.path.join(target_dir, folder))
    print(f"✅ Moved {folder} → {target_dir}")


✅ Moved ABIDEII-ABIDEII-BNI_1_29006 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29007 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29008 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29009 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29010 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29011 → D:\output\Non_Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29012 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29013 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29014 → D:\output\Non_Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29015 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29016 → D:\output\Non_Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29017 → D:\output\Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29018 → D:\output\Non_Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29019 → D:\output\Non_Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29020 → D:\output\Non_Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29021 → D:\output\Non_Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_29022 → D:\output\Non_Autism
✅ Moved ABIDEII-ABIDEII-BNI_1_290